# Evaluating ML Models

## Setup and Imports

In [1]:
import optuna

optuna.logging.set_verbosity(optuna.logging.WARN) #INFO, WARN

In [2]:
# import ray
# from ray import air, tune
# from ray.air import session
# from ray.tune import CLIReporter
# from ray.tune.schedulers import (ASHAScheduler,
#                                  PopulationBasedTraining,
#                                  HyperBandScheduler)
# from ray.tune.integration.pytorch_lightning import (TuneReportCallback,
#                                                     TuneReportCheckpointCallback)

# from ray.tune.search import ConcurrencyLimiter
# from ray.tune.search.optuna import OptunaSearch
# from ray.air import CheckpointConfig

In [3]:
from IPython.display import display_html
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = 'all'

import collections
import itertools
import re
import gc
import math
import numpy as np
import pandas as pd
import pickle
import requests as r
import matplotlib.pyplot as plt
import seaborn as sns
import shutil
import os

import typing
from typing import Mapping, Literal, Callable, List, ClassVar, Any, Tuple, Type

from uuid import uuid4
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs, MACCSkeys
from datetime import date
from scipy.sparse import csr_matrix, vstack

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import classification_report, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils import class_weight

pd.set_option('display.max_columns', 1000, 'display.width', 2000, 'display.max_colwidth', 100)

In [4]:
import xgboost as xgb

In [5]:
import pytorch_lightning as pl
from pytorch_lightning import LightningModule, Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import CSVLogger, TensorBoardLogger

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, random_split

from torchvision.ops import MLP

import torch_geometric
import torch_geometric.nn as geom_nn
import torch_geometric.data as geom_data
from torch_geometric.utils.smiles import from_smiles

from torchmetrics import (Accuracy,
                          AUROC,
                          ROC,
                          Precision,
                          Recall,
                          F1Score,
                          MeanAbsoluteError,
                          MeanSquaredError)
from torchmetrics.functional import (mean_absolute_error,
                                     mean_squared_error,
                                     mean_squared_log_error,
                                     pearson_corrcoef,
                                     r2_score)
from torchmetrics.functional.classification import (binary_accuracy,
                                                    binary_auroc,
                                                    binary_precision,
                                                    binary_recall,
                                                    binary_f1_score)

# Sets seeds for numpy, torch and python.random.
torch.manual_seed(42)
np.random.seed(42)
pl.seed_everything(42, workers=True)
# torch.use_deterministic_algorithms(True) # TODO: This is a GPU-related thing..

Global seed set to 42


42

Reduce logging information:

In [6]:
import logging
import warnings
import re

def set_global_logging_level(level=logging.ERROR, prefices=[""]):
    """
    Override logging levels of different modules based on their name as a prefix.
    It needs to be invoked after the modules have been loaded so that their loggers have been initialized.

    Args:
        - level: desired level. e.g. logging.INFO. Optional. Default is logging.ERROR
        - prefices: list of one or more str prefices to match (e.g. ["transformers", "torch"]). Optional.
          Default is `[""]` to match all active loggers.
          The match is a case-sensitive `module_name.startswith(prefix)`
    """
    prefix_re = re.compile(fr'^(?:{ "|".join(prefices) })')
    for name in logging.root.manager.loggerDict:
        if re.match(prefix_re, name):
            logging.getLogger(name).setLevel(level)

# Filter out annoying Pytorch Lightning printouts
warnings.filterwarnings('ignore', '.*does not have many workers.*')
warnings.filterwarnings('ignore', '.*Checkpoint directory.*')
warnings.filterwarnings('ignore', '.*The number of training batches.*')
logging.getLogger('pytorch_lightning').setLevel(logging.ERROR)
logging.getLogger("pytorch_lightning.utilities.rank_zero_warn").setLevel(logging.ERROR)
set_global_logging_level(logging.ERROR, ['transformers', 'nlp', 'torch', 'tensorflow', 'tensorboard', 'wandb', 'xgboost'])

Setup directories:

In [7]:
data_dir = os.path.join(os.getcwd(), '..', 'data')
src_dir = os.path.join(os.getcwd(), '..', 'src')
fig_dir = os.path.join(data_dir, 'figures')
# checkpoint_dir = os.path.join(os.getcwd(), '..', 'checkpoints')
checkpoint_dir = os.path.join(os.getcwd(), '..', 'checkpoints')
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
if not os.path.exists(src_dir):
    os.makedirs(src_dir)
if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

Load PROTAC-DB, used for training and validation:

In [8]:
df_file = os.path.join(data_dir, 'protac', 'protac-db_cleaned.csv')
protac_df = pd.read_csv(df_file)
protac_db_df = pd.concat([
    protac_df['Smiles'],
    protac_df['Smiles_nostereo'],
    protac_df['DC50'].astype(float),
    protac_df['pDC50'].astype(float),
    protac_df['Dmax'].astype(float),
    protac_df['poi_gene_id'],
    protac_df['poi_seq'],
    protac_df['cell_type'],
    protac_df['treatment_hours'],
    protac_df['active'],
], axis=1).reset_index(drop=True)
# protac_db_df['e3_ligase'] = protac_df['E3ligase']
print('protac_db_df: {:,} x {:,}'.format(*protac_db_df.shape))

protac_db_df: 429 x 10


Load PROTAC-Pedia, used for testing:

In [9]:
df_file = os.path.join(data_dir, 'protac', 'protac-pedia_cleaned.csv')
protac_df = pd.read_csv(df_file)
protac_pedia_df = pd.concat([
    protac_df['DC50'].astype(float),
    protac_df['pDC50'].astype(float),
    protac_df['Dmax'].astype(float),
    protac_df['poi_seq'],
    protac_df['cell_type'],
    protac_df['active'],
], axis=1).reset_index(drop=True)
protac_pedia_df['e3_ligase'] = protac_df['E3 Ligase']
protac_pedia_df['poi_gene_id'] = 'Unknown'
protac_pedia_df['Smiles'] = protac_df['PROTAC SMILES']
protac_pedia_df['Smiles_nostereo'] = protac_df['PROTAC SMILES_nostereo']
print('protac_pedia_df: {:,} x {:,}'.format(*protac_pedia_df.shape))

protac_pedia_df: 1,203 x 10


In [10]:
experiments_results = {}

In [50]:
def load_result(result_name):
    if not os.path.exists(os.path.join(checkpoint_dir, result_name + '.pkl')):
        print(f'WARNING: File {result_name} not found.')
        return None
    with open(os.path.join(checkpoint_dir, result_name + '.pkl'), 'rb') as fp:
        return pickle.load(fp)

def print_dict(title, d, filter_keys=True):
    print(f'{title}')
    filters = ['prediction', 'labels', 'logits', 'fpr', 'tpr', 'confusion']
    for k, v in d.items():
        if filter_keys:
            if not any([f in k for f in filters]):
                if isinstance(v, float):
                    if v < 1e-3:
                        print(f'\t* {k}: {v:.1e}')
                    else:
                        print(f'\t* {k}: {v:.3f}')
                else:
                    print(f'\t* {k}: {v}')
        else:
            print(f'\t* {k}: {v}')

In [88]:
experiments = [
    'xgb',
    'fp',
    'gnn',
    'transformer',
]
for experiment in experiments:
    result_type = f'results_{experiment}'
    experiments_results[result_type] = load_result(result_type)

df_data = []
df_data.append({
    'Type': 'Reference',
    'Models': 'Dummy',
    'val_acc': 0.5132,
    'val_roc_auc': 0.5,
    'test_acc': 0.5930,
    'test_roc_auc': 0.5,
})
df_data.append({
    'Type': 'Reference',
    'Models': 'DeepPROTACs',
    'val_acc': 0.7795,
    'val_roc_auc': 0.847,
    'test_acc': 0.,
    'test_roc_auc': 0.,
})
for k, results in experiments_results.items():
    for experiment_id, design_points in results.items():
        trial = design_points['trial']
        attrs = trial.user_attrs
        
        print_dict(f'{k} {experiment_id}:', trial.params)
        
        attrs['Experiment ID'] = experiment_id
        if 'xgb' in k:
            fp_bits = experiment_id[1]
            use_extra_features = experiment_id[-1]
            if use_extra_features:
                use_extra_features = ' w/ extra features'
            else:
                use_extra_features = ''
            fp_type = trial.params['fp_type'].replace('_fp', '').upper()
            attrs['Type'] = f'XGBoost'
            attrs['Models'] = f'XGBoost - {fp_type}-{fp_bits}b{use_extra_features}'
        if 'fp' in k:
            fp_bits = experiment_id[1]
            fp_type = trial.params['smiles_enc_kwargs_fp_type'].replace('_fp', '').upper()
            attrs['Models'] = f'MLP - {fp_type}-{fp_bits}b'
            attrs['Type'] = f'MLP'
        if 'gnn' in k:
            gnn_type = attrs['gnn_type']
            attrs['Models'] = f'GNN - {gnn_type[0].upper()}{gnn_type[1:-2]}{gnn_type[-2:].upper()}'
            attrs['Type'] = f'GNN'
        if 'transformer' in k:
            attrs['Models'] = f'BERT - {attrs["bert_type"]}'
            attrs['Type'] = f'Transformer'
        
        df_data.append(attrs)
        # print(f'Experiment: {experiment_id}')
        # print_dict('Hyperparams:', trial.params)
        # print_dict('Attributes:', trial.user_attrs)
        # print('-' * 80)
df = pd.DataFrame(df_data).astype(np.float64, errors='ignore')

results_xgb ('predict_active_inactive', 1024, False, True):
	* fp_type: maccs_fp
	* fp_radius: 5
	* fp_max_path: 10
	* booster: gbtree
	* lambda: 3.2e-07
	* alpha: 1.7e-08
	* max_depth: 15
	* eta: 0.034
	* gamma: 0.004
	* grow_policy: lossguide
results_xgb ('predict_active_inactive', 1024, False, False):
	* fp_type: morgan_fp
	* fp_radius: 2
	* fp_max_path: 9
	* booster: gblinear
	* lambda: 0.005
	* alpha: 5.1e-05
results_xgb ('predict_active_inactive', 2048, False, True):
	* fp_type: morgan_fp
	* fp_radius: 3
	* fp_max_path: 9
	* booster: dart
	* lambda: 8.7e-06
	* alpha: 6.1e-08
	* max_depth: 7
	* eta: 5.3e-04
	* gamma: 1.3e-07
	* grow_policy: depthwise
	* sample_type: uniform
	* normalize_type: forest
	* rate_drop: 0.053
	* skip_drop: 0.166
results_xgb ('predict_active_inactive', 2048, False, False):
	* fp_type: morgan_fp
	* fp_radius: 2
	* fp_max_path: 10
	* booster: gblinear
	* lambda: 1.1e-06
	* alpha: 2.1e-06
results_xgb ('predict_active_inactive', 4096, False, True):
	* fp_type

In [ ]:
# Convert all tensor entries into float values
for col in df.columns:
    try:
        df[col] = df[col].apply(lambda x: x.item() if isinstance(x, torch.Tensor) else x)
    except:
        pass

df[['val_acc', 'test_acc']] *= 100
display(df[['Experiment ID', 'Type', 'Models', 'val_acc', 'val_roc_auc', 'test_acc', 'test_roc_auc']])

,Experiment ID,Type,Models,val_acc,val_roc_auc,test_acc,test_roc_auc
0,NaN,Reference,Dummy,51.320000,0.500000,59.300000,0.500000
1,NaN,Reference,DeepPROTACs,77.950000,0.847000,0.000000,0.000000
2,"(predict_active_inactive, 1024, False, True)",XGBoost,XGBoost - MACCS-1024b w/ extra features,81.578946,0.826057,56.976742,0.709524
3,"(predict_active_inactive, 1024, False, False)",XGBoost,XGBoost - MORGAN-1024b,76.315790,0.797990,51.162791,0.576471
4,"(predict_active_inactive, 2048, False, True)",XGBoost,XGBoost - MORGAN-2048b w/ extra features,73.684210,0.861053,53.488374,0.569188
5,"(predict_active_inactive, 2048, False, False)",XGBoost,XGBoost - MORGAN-2048b,80.263156,0.813583,48.837209,0.501961
6,"(predict_active_inactive, 4096, False, True)",XGBoost,XGBoost - MACCS-4096b w/ extra features,81.578946,0.826057,56.976742,0.698319
7,"(predict_active_inactive, 4096, False, False)",XGBoost,XGBoost - PATH-4096b,73.684210,0.827789,61.627907,0.627451
8,"(predict_active_inactive, 1024, False)",MLP,MLP - MORGAN-1024b,75.000000,0.818087,55.813956,0.606723
9,"(predict_active_inactive, 2048, False)",MLP,MLP - MORGAN-2048b,75.000000,0.796951,67.441863,0.677311


In [ ]:
metrics = [
    ('Validation accuracy (%)', 'val_acc'),
    ('Test accuracy (%)', 'test_acc'),
    ('Validation AUC', 'val_roc_auc'),
    ('Test AUC', 'test_roc_auc'),
]
for title, metric in metrics:
    ax = sns.barplot(data=df, y='Models', x=metric, orient='h', palette='turbo')
    if 'acc' in metric:
        ax.set_xlim(0., 100.)
        fmt = '%.2f'
    else:
        ax.set_xlim(0., 1.)
        fmt = '%.2f'
        # plt.grid(axis='x', alpha=0.5)

    for bars_group in ax.containers:
        ax.bar_label(bars_group, padding=5, fmt=fmt) # fontsize=12
    
    dummy_score = float(df[df['Models'] == 'Dummy'][metric])
    ax.axvline(dummy_score, ls='--', c='black', alpha=0.3)
    
    ref_score = float(df[df['Models'] == 'DeepPROTACs'][metric])
    if ref_score > 0:
        ax.axvline(ref_score, ls='--', c='black', alpha=0.3)
    
    # plt.xticks(rotation=90)
    plt.title(title)
    # plt.legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=1) #, fancybox=True, shadow=True)
    plt.savefig(os.path.join(fig_dir, f'barplot_{metric}.pdf'))
    plt.savefig(os.path.join(fig_dir, f'barplot_{metric}.png'))
    plt.show()

In [ ]:
val_df = df.copy()
test_df = df.copy()

# Copy 'val_acc' and 'val_roc_auc' columns to 'acc' and 'roc_auc' columns respectively
val_df['Accuracy'] = df['val_acc']
val_df['AUC'] = df['val_roc_auc']
val_df['Dataset'] = 'Validation'

test_df['Accuracy'] = df['test_acc']
test_df['AUC'] = df['test_roc_auc']
test_df['Dataset'] = 'Test'

new_df = pd.concat([val_df, test_df])
# Reset the index of the new dataframe
new_df.reset_index(drop=True, inplace=True)

# Print the resulting dataframe
# display(new_df)

# plt.figure(figsize=(6, 10))
# ax = sns.barplot(data=new_df, x='Accuracy', y='Models', hue='Dataset', orient='h')
# ax.set_xlim(0., 100.)
# plt.grid(axis='x', alpha=0.8)

plt.figure(figsize=(12, 4))
ax = sns.barplot(data=new_df, y='Accuracy', x='Models', hue='Dataset')
ax.set_ylim(0., 100.)
plt.grid(axis='y', alpha=0.8)
plt.xticks(rotation=90)

dummy_score = float(df[df['Models'] == 'Dummy']['test_acc'])
# ax.axhline(dummy_score, ls='--', c='black', alpha=0.3)
ax.axhline(dummy_score, ls='--', c='black', alpha=0.99, label='Dummy model\ntest accuracy')

fmt = '%.2f'
fmt = '{:.1f}'
for bars_group in ax.containers:
    # Add note for DeepPROTACs test results not being available
    labels = ['(Not available)' if x == 0 else f'{x:,.1f}' for x in bars_group.datavalues]
    ax.bar_label(bars_group, padding=5, fmt=fmt, labels=labels, rotation=90) # fontsize=12
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=1) #, fancybox=True, shadow=True)
plt.title(f'Accuracy results (%)')
f = os.path.join(fig_dir, f'barplot_accuracy')
plt.savefig(f + '.pdf', bbox_inches='tight')
plt.savefig(f + '.png', bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=(12, 4))
ax = sns.barplot(data=new_df, y='AUC', x='Models', hue='Dataset')
ax.set_ylim(0., 1.)
plt.grid(axis='y', alpha=0.8)
plt.xticks(rotation=90)

dummy_score = float(df[df['Models'] == 'Dummy']['test_roc_auc'])
# ax.axhline(dummy_score, ls='--', c='black', alpha=0.3)
ax.axhline(dummy_score, ls='--', c='black', alpha=0.99, label='Dummy model\ntest AUC')

fmt = '%.2f'
for bars_group in ax.containers:
    labels = ['(Not available)' if x == 0 else f'{x:,.2f}' for x in bars_group.datavalues]
    ax.bar_label(bars_group, padding=5, fmt=fmt, labels=labels, rotation=90) # fontsize=12
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=1) #, fancybox=True, shadow=True)
plt.title(f'AUC results')
f = os.path.join(fig_dir, f'barplot_auc')
plt.savefig(f + '.pdf', bbox_inches='tight')
plt.savefig(f + '.png', bbox_inches='tight')
plt.show()

In [ ]:
new_df = df.copy()
new_df['Accuracy Drop'] = df['val_acc'] - df['test_acc']
new_df['AUC Drop'] = df['val_roc_auc'] - df['test_roc_auc']

# Drop Dummy and DeepPROTACs rows
new_df = new_df[new_df['Models'] != 'Dummy']
new_df = new_df[new_df['Models'] != 'DeepPROTACs']
# Reset the index of the new dataframe
new_df.reset_index(drop=True, inplace=True)

# Print the resulting dataframe
# plt.figure(figsize=(12, 4))
ax = sns.barplot(data=new_df, y='Accuracy Drop', x='Type')
# ax.set_ylim(0., 100.)
plt.grid(axis='y', alpha=0.8)
plt.xticks(rotation=90)


fmt = '{:.1f}%'
for bars_group in ax.containers:
    ax.bar_label(bars_group, padding=1, fmt=fmt, rotation=0) # fontsize=12
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=1) #, fancybox=True, shadow=True)
plt.title(f'Accuracy drops (%)')

f = os.path.join(fig_dir, f'barplot_accuracy_drops_per_type')
plt.savefig(f + '.pdf', bbox_inches='tight')
plt.savefig(f + '.png', bbox_inches='tight')
plt.show()

In [ ]:
new_df = df.copy()
new_df['Accuracy Drop'] = df['val_acc'] - df['test_acc']
new_df['AUC Drop'] = df['val_roc_auc'] - df['test_roc_auc']

# Drop Dummy and DeepPROTACs rows
new_df = new_df[new_df['Models'] != 'Dummy']
new_df = new_df[new_df['Models'] != 'DeepPROTACs']
# Reset the index of the new dataframe
new_df.reset_index(drop=True, inplace=True)

# Print the resulting dataframe
plt.figure(figsize=(10, 4))
ax = sns.barplot(data=new_df, y='Accuracy Drop', x='Models')
# ax.set_ylim(0., 100.)
plt.grid(axis='y', alpha=0.8)
plt.xticks(rotation=90)


fmt = '{:.1f}%'
for bars_group in ax.containers:
    ax.bar_label(bars_group, padding=1, fmt=fmt, rotation=0) # fontsize=12
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=1) #, fancybox=True, shadow=True)
plt.title(f'Accuracy drops (%)')

f = os.path.join(fig_dir, f'barplot_accuracy_drops_per_model')
plt.savefig(f + '.pdf', bbox_inches='tight')
plt.savefig(f + '.png', bbox_inches='tight')
plt.show()

In [ ]:
df_data = []
for k, results in load_result('results_transformer_ssl').items():
    print_dict(k, results)
    df_data.append({'BERT Model': k.replace('SSL_', ''), 'Perplexity on SSL set': 'Before finetuning', 'Perplexity score': results['train_perplexity_before']})
    df_data.append({'BERT Model': k.replace('SSL_', ''), 'Perplexity on SSL set': 'After finetuning', 'Perplexity score': results['train_perplexity_after']})
    df_data.append({'BERT Model': k.replace('SSL_', ''), 'Perplexity on Val set': 'Before finetuning', 'Perplexity score': results['val_perplexity_before']})
    df_data.append({'BERT Model': k.replace('SSL_', ''), 'Perplexity on Val set': 'After finetuning', 'Perplexity score': results['val_perplexity_after']})
df = pd.DataFrame(df_data).astype(np.float64, errors='ignore')

for hue in ['Perplexity on SSL set', 'Perplexity on Val set']:
    plt.figure(figsize=(8, 4.5))
    ax = sns.barplot(data=df, y='Perplexity score', x='BERT Model', hue=hue)
    # ax.set_ylim(0., 100.)
    ax.set_yscale('log')
    plt.grid(axis='y', alpha=0.8)
    # plt.xticks(rotation=90)

    fmt = '%.2f'
    for bars_group in ax.containers:
        ax.bar_label(bars_group, padding=0.5, fmt=fmt) # fontsize=12
    # plt.legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=1) #, fancybox=True, shadow=True)
    plt.title(hue)
    plt.show()

## DataFrames To LateX

In [ ]:
# import pandas as pd
# from io import StringIO

# csv = '''
# Experiment ID	Type	Models	val_acc	val_roc_auc	test_acc	test_roc_auc
# NaN	Reference	Dummy	51.320000	0.500000	59.300000	0.500000
# NaN	Reference	DeepPROTACs	77.950000	0.847000	0.000000	0.000000
# (predict_active_inactive, 1024, False, True)	XGBoost	XGBoost - MACCS-1024b w/ extra features	81.578946	0.826057	56.976742	0.709524
# (predict_active_inactive, 1024, False, False)	XGBoost	XGBoost - MORGAN-1024b	76.315790	0.797990	51.162791	0.576471
# (predict_active_inactive, 2048, False, True)	XGBoost	XGBoost - MORGAN-2048b w/ extra features	73.684210	0.861053	53.488374	0.569188
# (predict_active_inactive, 2048, False, False)	XGBoost	XGBoost - MORGAN-2048b	80.263156	0.813583	48.837209	0.501961
# (predict_active_inactive, 4096, False, True)	XGBoost	XGBoost - MACCS-4096b w/ extra features	81.578946	0.826057	56.976742	0.698319
# (predict_active_inactive, 4096, False, False)	XGBoost	XGBoost - PATH-4096b	73.684210	0.827789	61.627907	0.627451
# (predict_active_inactive, 1024, False)	MLP	MLP - MORGAN-1024b	75.000000	0.818087	55.813956	0.606723
# (predict_active_inactive, 2048, False)	MLP	MLP - MORGAN-2048b	75.000000	0.796951	67.441863	0.677311
# (predict_active_inactive, 4096, False)	MLP	MLP - MORGAN-4096b	80.263156	0.814969	54.651165	0.586275
# (predict_active_inactive, False, attentivefp)	GNN	GNN - AttentiveFP	69.736844	0.709979	50.000000	0.501961
# (predict_active_inactive, False, gat)	GNN	GNN - GAT	68.421054	0.743243	44.186047	0.463305
# (predict_active_inactive, False, gcn)	GNN	GNN - GCN	76.315790	0.744629	62.790698	0.633053
# (predict_active_inactive, False, gin)	GNN	GNN - GIN	72.368419	0.708247	60.465115	0.625210
# (predict_active_inactive, False, entropy/roberta_zinc_480m)	Transformer	BERT - roberta_zinc_480m	72.368419	0.866597	55.813956	0.566667
# (predict_active_inactive, False, seyonec/ChemBERTa-zinc-base-v1)	Transformer	BERT - ChemBERTa-zinc-base-v1	78.947371	0.863825	63.953489	0.624370
# (predict_active_inactive, False, DeepChem/ChemBERTa-10M-MTR)	Transformer	BERT - ChemBERTa-10M-MTR	76.315790	0.808039	63.953489	0.614566
# (predict_active_inactive, False, SSL_roberta_zinc_480m)	Transformer	BERT - SSL_roberta_zinc_480m	76.315790	0.833333	61.627907	0.644538
# (predict_active_inactive, False, SSL_ChemBERTa-zinc-base-v1)	Transformer	BERT - SSL_ChemBERTa-zinc-base-v1	77.631581	0.818780	59.302324	0.609804
# (predict_active_inactive, False, SSL_ChemBERTa-10M-MTR)	Transformer	BERT - SSL_ChemBERTa-10M-MTR	73.684210	0.812197	63.953489	0.587675
# '''

# df = pd.read_csv(StringIO(csv), sep='\t')
# display(df)

In [112]:
old2new = {
    'val_acc': 'Accuracy [%] (Validation)',
    'test_acc': 'Accuracy [%] (Test)',
    'val_roc_auc': 'ROC AUC (Validation)',
    'test_roc_auc': 'ROC AUC (Test)',
}
for k, results in experiments_results.items():
    print('-' * 80)
    print(k)
    print('-' * 80)
    tmp = []
    for experiment_id, design_points in results.items():
        params = design_points['trial'].params.copy()
        params.update({q: design_points['trial'].user_attrs[q] for q in old2new.keys()})
        params['val_acc'] *= 100.0
        params['test_acc'] *= 100.0
        params['layer_sizes'] = []
        for k, v in params.items():
            if 'layer' in k and 'size' in k and 'smiles' not in k and 'layer_sizes' not in k:
                params['layer_sizes'].append(v)
        if 'xgb' in k:
            params['extra_features (fixed)'] = experiment_id[-1]
        elif 'fp' in k:
            params['fp_bits (fixed)'] = experiment_id[1]
        elif 'gnn' in k:
            params['gnn_type (fixed)'] = experiment_id[-1]
        elif 'transformer' in k:
            params['bert_type (fixed)'] = experiment_id[-1].split('/')[-1]
        tmp.append(params)
        # print_dict(f'{k} {experiment_id}:', trial.params)
    tmp = pd.DataFrame(tmp).fillna('-')
    tmp = tmp.rename(columns=old2new)
    # print(tmp.T.to_latex(escape=True))
    print(tmp.T.to_latex(escape=True, float_format='{:4.1g}'.format))

--------------------------------------------------------------------------------
results_xgb
--------------------------------------------------------------------------------
\begin{tabular}{lllllll}
\toprule
{} &                   0 &                   1 &                   2 &                   3 &                   4 &                   5 \\
\midrule
fp\_type                   &            maccs\_fp &           morgan\_fp &           morgan\_fp &           morgan\_fp &            maccs\_fp &             path\_fp \\
fp\_radius                 &                   5 &                   2 &                   3 &                   2 &                  10 &                   7 \\
fp\_max\_path               &                  10 &                   9 &                   9 &                  10 &                  10 &                  10 \\
booster                   &              gbtree &            gblinear &                dart &            gblinear &                dart &               

C:\Users\ste\AppData\Local\Temp\ipykernel_27180\920848393.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(tmp.T.to_latex(escape=True, float_format='{:4.1g}'.format))
C:\Users\ste\AppData\Local\Temp\ipykernel_27180\920848393.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(tmp.T.to_latex(escape=True, float_format='{:4.1g}'.format))
C:\Users\ste\AppData\Local\Temp\ipykernel_27180\920848393.py:34: FutureWarning: In future versions `DataFrame.to_latex` is e

In [64]:
tmp = df[df['Type'] == 'XGBoost'].astype(np.float64, errors='ignore')
# tmp = tmp.dropna()

old2new = {
    'val_acc': 'Accuracy [%] (Validation)',
    'test_acc': 'Accuracy [%] (Test)',
    'val_roc_auc': 'ROC AUC (Validation)',
    'test_roc_auc': 'ROC AUC (Test)',
}
tmp = tmp.rename(columns=old2new)

# def polish(row):
#     if 'w/ extra features' in row['Models']:
#         row['extra_features (fixed)'] = True
#     else:
#         row['extra_features (fixed)'] = False
#     if '1024' in row['Models']:
#         row['fp_bits (fixed)'] = 1024
#     if '2048' in row['Models']:
#         row['fp_bits (fixed)'] = 2048
#     if '4096' in row['Models']:
#         row['fp_bits (fixed)'] = 4096
#     if 'MACCS' in row['Models']:
#         row['fp_bits (fixed)'] = 167
#         row['fp_type'] = 'MACCS'
#     if 'MORGAN' in row['Models']:
#         row['fp_type'] = 'Morgan'
#     if 'PATH' in row['Models']:
#         row['fp_type'] = 'Path'
#     return row

# tmp = tmp.apply(polish, axis=1)

# print(tmp.T.style.to_latex())
# print('\n\n')

print(tmp.T.to_latex(escape=True, float_format='{:0.2f}'.format))
print('\n\n')
print(tmp.T.to_latex(escape=True, float_format='{:0.e}'.format))

results_xgb ('predict_active_inactive', 1024, False, True):
	* fp_type: maccs_fp
	* fp_radius: 5
	* fp_max_path: 10
	* booster: gbtree
	* lambda: 3.2e-07
	* alpha: 1.7e-08
	* max_depth: 15
	* eta: 0.034
	* gamma: 0.004
	* grow_policy: lossguide
results_xgb ('predict_active_inactive', 1024, False, False):
	* fp_type: morgan_fp
	* fp_radius: 2
	* fp_max_path: 9
	* booster: gblinear
	* lambda: 0.005
	* alpha: 5.1e-05
results_xgb ('predict_active_inactive', 2048, False, True):
	* fp_type: morgan_fp
	* fp_radius: 3
	* fp_max_path: 9
	* booster: dart
	* lambda: 8.7e-06
	* alpha: 6.1e-08
	* max_depth: 7
	* eta: 5.3e-04
	* gamma: 1.3e-07
	* grow_policy: depthwise
	* sample_type: uniform
	* normalize_type: forest
	* rate_drop: 0.053
	* skip_drop: 0.166
results_xgb ('predict_active_inactive', 2048, False, False):
	* fp_type: morgan_fp
	* fp_radius: 2
	* fp_max_path: 10
	* booster: gblinear
	* lambda: 1.1e-06
	* alpha: 2.1e-06
results_xgb ('predict_active_inactive', 4096, False, True):
	* fp_type

C:\Users\ste\AppData\Local\Temp\ipykernel_27180\2478884251.py:46: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(tmp.T.to_latex(escape=True, float_format='{:0.2f}'.format))
C:\Users\ste\AppData\Local\Temp\ipykernel_27180\2478884251.py:48: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(tmp.T.to_latex(escape=True, float_format='{:0.ef}'.format))


ValueError: Format specifier missing precision

## Optuna Plots

In [ ]:
figsize = (5, 5)
plt.figure(figsize=figsize)
opt_point = (0.45, 0.7)
rand_space = np.random.random(size=(100, 2))
plt.scatter(rand_space[:, 0], rand_space[:, 1], label='Searched points')
plt.scatter(*opt_point, marker='x', c='r', label='Optimal point')
plt.title('Random search')
plt.xlabel('Hyperparameter 1')
plt.ylabel('Hyperparameter 2')
plt.legend()
plt.grid(alpha=0.8)
plt.show()

plt.figure(figsize=figsize)
for i in range(10):
    x = np.random.default_rng().uniform(opt_point[0] - i * 0.1, opt_point[0] + i * 0.1, max(1, 20 - i * 2))
    y = np.random.default_rng().uniform(opt_point[1] - i * 0.1, opt_point[1] + i * 0.1, max(1, 20 - i * 2))
    plt.scatter(x, y, c='C0', label='Searched points' if i == 0 else '')
# plt.scatter(rand_space[:, 0], rand_space[:, 1], c='C0', label='Searched points')
rand_space = np.random.random(size=(5, 2))

plt.scatter(*opt_point, marker='x', c='r', label='Optimal point')
plt.title('Optimized Optuna search')
plt.xlabel('Hyperparameter 1')
plt.ylabel('Hyperparameter 2')
plt.legend()
plt.xlim(0., 1.)
plt.ylim(0., 1.)
plt.grid(alpha=0.8)
plt.show()